In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# 使用FLASK创建应用：自动作诗
> <img style="width:200px; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/poet.png"/>

        诗仙 1.0


# 1.概述

> <img style="width:70%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/flask-poet.png"/>

    在上节课的基础上，增加如何调用业务处理的逻辑过程，各个模块各司其职：
        动态页面 - Flask Jinja
        访问/作诗计数器 - Redis 缓存
        诗词自动生成 - Fortune-zh


# 2.code
> 完整代码：[下载](https://resource.xtalker.cn:8000/_static/code/flask_poet.zip)

    创建自己的项目目录(任何你喜欢的目录名均可，以后就是 Docker-Compose的项目名)：
        flask_poet
    镜像开发所需要准备的代码文件：
        build/requirements.txt
        build/debian.sources
        build/pip.conf
        build/Dockerfile
    项目启动运行所需要准备的代码文件：
        code/app.py
        code/templates/poet.html
        code/static/flask-poet.png
        compose.yml
    系统配置（不是必须）：
        Windows默认位置：C:\Windows\System32\drivers\etc\hosts
        Linux：/etc/hosts


## 2.1.镜像的开发

### build/Dockerfile
    能看懂如下的代码吗？不懂就问
    
```shell
FROM python:slim-bookworm

COPY . /root/files
# 使用国内安装源，加快下载速度
RUN mv /root/files/debian.sources /etc/apt/sources.list.d/debian.sources; \
apt update; \
# 安装认证中心、语言设置工具和fortunes中文库
apt install -y ca-certificates locales net-tools fortunes-zh; \
# 设置中文语言支持环境
localedef -c -f UTF-8 -i zh_CN zh_CN.utf8; \
# 整理中文数据库，只保留和生成唐诗宋词
rm /usr/share/games/fortunes/chinese*

# 中文环境变量
ENV LANG="zh_CN.UTF-8" LANGUAGE="zh_CN:zh"  LC_ALL="zh_CN.UTF-8"

# 使用国内安装源，加快pip下载安装速度
WORKDIR /code
RUN mv /root/files/pip.conf /etc/pip.conf; \
mv /root/files/requirements.txt requirements.txt; \
pip install -r requirements.txt

CMD ["flask", "run"]
```

###  build/requirements.txt
    略
    
###  build/pip.conf
    用于加速 pip 安装下载速度

### build/debian.sources
    用于加速 apt-get 安装下载速度

## 2.2.模板开发

### code/templates/poet.html
    整个的表现逻辑如下，而业务逻辑封装在 {{ ... }}
    
```html
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
{{ fortune }}
赏析 {{ hits }} 次数
</pre>
<img style="width:200px; display: inline-block" src="static/flask-poet.png"/>
```

## 2.3.Python代码
    利用 Flask 框架可以非常容易构建应用
### app.py
    代码解析，其中 subprocess 用于调用 shell 命令
```python
import time
import subprocess  

import redis
from flask import Flask, render_template, request
app = Flask(__name__)

# 启用调试模式
if __name__ == '__main__':
    app.run(debug=True)

cache = redis.Redis(host='redis', port=6379)

# 执行 shell 命令  
def get_fortune():
    result = subprocess.run(['/usr/games/fortune'], capture_output=True, text=True)
    # 检查是否有错误  
    if result.returncode != 0:
        return f"Error occurred: {result.stderr}"
    else:
        return result.stdout

# 利用 Redis 缓存机制实现计数器
def get_hit_count():
    retries = 5
    while True:
        try:
            return cache.incr('hits')
        except redis.exceptions.ConnectionError as exc:
            if retries == 0:
                raise exc
            retries -= 1
            time.sleep(0.5)

# 调用模板渲染功能生成网页
@app.route('/')
def fortune():
    fortune = get_fortune()
    count = get_hit_count()
    return render_template("poet.html",fortune=fortune,hits=count)
```

## 2.4.应用部署
### compose.yml

```yaml
services:
  # 应用容器实例
  web:
    image: poet:1.0
    build:
      context: "build"
      dockerfile: Dockerfile
    restart: "unless-stopped"
    tty: true
    networks:
      - python-net
    hostname: web
    ports:
     - "80:5000"
    environment:
      - FLASK_APP=app.py
      - FLASK_RUN_HOST=0.0.0.0
    volumes:
      # 将目录映射到容器，包含代码、模版和图片资源
      - .\code:/code
    depends_on:
      - redis
  
  # Redis数据容器实例
  redis:
    image: redis:alpine
    networks:
      - python-net

networks:
  python-net:
    name: python-br
```

### hosts
    hosts:
        C:\Windows\System32\drivers\etc\hosts
    用文本编辑器以管理员模式打开hosts文件，在文件的结尾处增加一条记录，如下所示：
    
```ini
# 你也可以使用任何你喜欢的域名来替代 poet.org
127.0.0.1 poet.org www.poet.org
```
    测试一下是否生效：
```shell
ping poet.org
# 生效，输出如下：
正在 Ping hello.world [127.0.0.1] 具有 32 字节的数据:
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
# 未生效
Ping 请求找不到主机 poet.org。请检查该名称，换一个域名然后重试。
```

# 3.test
## 命令执行过程

```shell
cd flask_poet

# 构建
docker-compose build
...

# 创建并启动
docker-compose up -d
[+] Running 2/0
 ✔ Container flask_poet-redis-1  Running                                                                           0.0s
 ✔ Container flask_poet-web-1    Running


# 构建成功会在主机生成镜像：poet:1.0
docker-compose images
CONTAINER            REPOSITORY          TAG                 IMAGE ID            SIZE
flask_poet-redis-1   redis               alpine              f597a450f464        40.7MB
flask_poet-web-1     poet                1.0                 a4601a9314ff        192MB

# 测试
curl http://www.poet.org/
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
《贾生》
作者：李商隐
宣室求贤访逐臣，贾生才调更无伦。
可怜夜半虚前席，不问苍生问鬼神。

赏析 1 次数
</pre>
<img style="width:200px; display: inline-block" src="static/flask-poet.png"/>
```
## 浏览器访问 http://www.poet.org
> <img style="width:50%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/flask-poet-demo.png"/>


# 4.课后作业
    1.请用 Flask 实现自动作诗的主页，并计数
    2.思考一下，如果你做应用项目将按照什么步骤进行